In [1]:
import numpy as np
import pandas as pd

import lightgbm as lgb
import optuna
import pickle
import seaborn as sns

In [2]:
# Import my modules.
import sys, os
from pathlib import Path
current_dir = os.path.join(Path().resolve())
sys.path.append(str(current_dir) + '/../')

from modules import utils

In [3]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [4]:
# Parameters
USE_LAG = 5
USE_OLD_LOG = False
USE_TREND=True

## Import data and create basic features

In [5]:
BASE = '../input/'
date_col = 'first_day_of_month'
cat_cols = ['county', 'state']
mbd = 'microbusiness_density'
idx = 'row_id'

In [6]:
df_census = pd.read_csv(BASE + 'census_starter.csv', index_col='cfips')
df_train = pd.read_csv(BASE + 'train.csv',  index_col=idx)
df_test = pd.read_csv(BASE + 'test.csv',  index_col=idx)
df_subm = pd.read_csv(BASE + 'sample_submission.csv',  index_col=idx)

In [7]:
state_dict = df_train[['cfips', 'state', 'county']]
state_dict = state_dict.set_index('cfips')
state_dict = state_dict.drop_duplicates()
state_dict = state_dict.to_dict()

df_test['state'] = df_test['cfips'].map(state_dict['state'])
df_test['county'] = df_test['cfips'].map(state_dict['county'])

df_all = pd.concat([df_train, df_test], axis=0)

df_all[date_col] = pd.to_datetime(df_all[date_col])

df_all['year'] = df_all[date_col].dt.year
df_all['month'] = df_all[date_col].dt.month
df_all['scale'] = (df_all[date_col] - df_all[date_col].min()).dt.days
df_all['scale'] = df_all['scale'].factorize()[0]

df_all = df_all.drop(columns=[date_col])
df_all.sort_index(inplace=True)

df_all[cat_cols] = df_all[cat_cols].astype('category')

# df_all.to_csv('../output/df_all.csv')

## Join features created by other codes

In [8]:
# df_feature_season = pd.read_csv('../output/feature_season.csv')

# df_all.reset_index(inplace=True)
# df_all = df_all.merge(df_feature_season, how='left', on='row_id')
# df_all.set_index('row_id', inplace=True)

# df_all.head()

## Create features for modeling

In [9]:
for i in range(30, 39):
    dt = df_all.loc[df_all.scale==i].groupby('cfips')['active'].agg('last')
    df_all[f'select_lastactive{i}'] = df_all['cfips'].map(dt)

    dt = df_all.loc[df_all.scale==i].groupby('cfips')['microbusiness_density'].agg('last')
    df_all[f'select_lastmbd{i}'] = df_all['cfips'].map(dt)

In [10]:
for i in range(1, 5):
    df_all[f'select_rate{i}'] = df_all.groupby('cfips')[mbd].shift(i).bfill()
    df_all[f'select_rate{i}'] = (df_all[mbd] / df_all[f'select_rate{i}'] - 1).fillna(0)

In [11]:
for i in range(1, 5):
    for j in range(i, i + USE_LAG):
        df_all[f'select_rate{i}_lag{j}'] = df_all[f'select_rate{i}'].shift(j) 

# df_all.to_csv('../output/df_all_lag.csv')

In [12]:
df_all = df_all.reset_index()
df_all = df_all.set_index('cfips')

df_all[df_census.columns] = df_census

df_all = df_all.reset_index()
df_all = df_all.set_index('row_id')

## Run Validation

In [13]:
output_features = ['cfips', 'county', 'state', 'microbusiness_density', 'active', 'year','month', 'scale', 
                                 'mbd_pred', 'y_base', 'y_pred', 'y_target', 'smape']

def regularize(x):
    if x >= 1:
        if x * 0.999 >= 1:
            x *= 0.999
        else:
            x = 1
    else:
        if x * 1.001 <= 1:
            x *= 1.001
        else:
            x = 1
    return x

def get_trend_dict(valid_time, pred_m = 1, n=3, thre=2, active_thre=25000):
    target=mbd
    df_target_lag = df_all.loc[(df_all['scale'] >= valid_time - pred_m - n)&(df_all['scale']<=valid_time-pred_m), ['cfips','scale','active',target]].copy()
    # df_target_lag[target] += 1 # これは本来は不要だった
    for i in range(1, n+1):
        df_target_lag[f'lag_{i}'] = df_target_lag[target].shift(i)

    for i in range(1, n+1):
        if i==1:
            df_target_lag[f'rate{i}'] = df_target_lag[target] / df_target_lag[f'lag_{i}']
        else:
            df_target_lag[f'rate{i}'] = df_target_lag[f'lag_{i-1}'] / df_target_lag[f'lag_{i}']        

    df_target_lag['up_cnt'] = 0
    df_target_lag['down_cnt'] = 0
    df_target_lag['mean'] = 0
    for i in range(1, n+1):
        df_target_lag['up_cnt'] += (df_target_lag[f'rate{i}'] > 1)*1
        df_target_lag['down_cnt'] += (df_target_lag[f'rate{i}']<1)*1
        df_target_lag['mean'] += df_target_lag[f'rate{i}']
    df_target_lag['mean'] /= n

    df_target_lag['trend'] = df_target_lag[['up_cnt', 'mean']].apply(lambda x: x[1] if x[0] >= thre and x[1]>1 else np.nan, axis=1)
    df_target_lag['trend'] = df_target_lag[['down_cnt', 'mean', 'trend']].apply(lambda x: x[1] if x[0] >= thre and x[1]<1 else x[2], axis=1)
    idx = (df_target_lag['scale']==valid_time-pred_m)&(df_target_lag['active']>=active_thre)&(~df_target_lag['trend'].isna())
    df_trend = df_target_lag[idx].copy()
    #df_trend['trend'] = df_trend['trend'].apply(regularize)
    df_trend['trend'] = df_trend['trend'].clip(0.995, 1.005)
    trend_dict = df_trend[['cfips', 'trend']].set_index('cfips').to_dict()['trend']
    
    return trend_dict


def run_fit_predict(valid_time, pred_m):

    train_times = valid_time - pred_m
    
    print('valid_times: ', valid_time)
    print('pred_m: ', pred_m)
    print('train_times: ', train_times)

    drop_features = ['microbusiness_density', 'active', 'scale']
    features = list(filter(lambda x: (not x.startswith('select_') and (x not in drop_features)),  df_all.columns.to_list()))
    
    # Select appropriate lastactive and lastmbd features.
    features.append(f'select_lastactive{train_times}')
    features.append(f'select_lastmbd{train_times}')
    
    # Select appropriate target and lag features.
    target = f'select_rate{pred_m}'
    for i in range(pred_m, pred_m + USE_LAG):
        features.append(f'select_rate{pred_m}_lag{i}')

    # Extract Valid and Train data.
    if USE_OLD_LOG:
        train_indices = (df_all['scale']<=train_times) & (df_all['scale']>=pred_m)
    else:
        train_indices = (df_all['scale']<=train_times) & (df_all['scale']>=pred_m + USE_LAG)

    X_train = df_all.loc[train_indices, features]
    y_train = df_all.loc[train_indices, target]

    valid_indices = (df_all['scale']==valid_time)
    X_valid = df_all.loc[valid_indices, features]
    y_valid = df_all.loc[valid_indices, target]
    
    # Create Model and predict.
    params = {
        'n_iter': 200,
        'verbosity': -1,
        'objective': 'l1',
        'random_state': 42,
        'extra_trees': True,
        'colsample_bytree': 0.8841279649367693,
        'colsample_bynode': 0.10142964450634374,
        'max_depth': 8,
        'learning_rate': 0.013647749926797374,
        'lambda_l1': 1.8386216853616875,
        'lambda_l2': 7.557660410418351,
        'num_leaves': 61,
        'min_data_in_leaf': 213
    }
    model = lgb.LGBMRegressor(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    
    # Convert y_pred to microbusiness_density prediction and create output dataset.
    df_valid = df_all.loc[valid_indices].copy()
    df_valid['y_pred'] = y_pred
    df_valid['y_target'] = y_valid
    base_indices = (df_all['scale']==valid_time-pred_m)
    base_y = df_all.loc[base_indices, ['cfips', 'microbusiness_density']]
    base_dict = base_y.set_index('cfips').to_dict()
    df_valid['y_base'] = df_valid['cfips'].map(base_dict['microbusiness_density'])
    df_valid['mbd_pred'] = df_valid['y_base'] * (df_valid['y_pred']+1)
    
    if USE_TREND and pred_m == 1:
        trend_dict = get_trend_dict(valid_time, pred_m, 3, 3, 5000)
        print('# of cfips that have trend :', len(trend_dict))
        for cfip in trend_dict:
            df_valid.loc[df_valid['cfips']==cfip, 'mbd_pred'] = df_valid.loc[df_valid['cfips']==cfip, 'y_base'] * trend_dict[cfip]
    
    df_valid['smape'] = utils.smape_arr(df_valid['microbusiness_density'], df_valid['mbd_pred'])
    df_output = df_valid[output_features]
    
    return df_output


def run_validation_for_pred_m(validation_times, pred_ms):
    
    df_output = pd.DataFrame(columns=output_features)
    for validation_time, pred_m in zip(validation_times, pred_ms):
        df = run_fit_predict(validation_time, pred_m)
        df_output = pd.concat([df, df_output])

    return df_output.reset_index().rename(columns={'index': 'row_id'}).set_index('row_id')


def run_validation(max_month=38, m_len=5, pred_ms = [1,2,3,4]):
    
        validation_times = [max_month - i for i in range(m_len)]
        
        output_dic = dict()
        for pred_m in pred_ms:
            pred_m_len = [pred_m] * m_len
            df_output = run_validation_for_pred_m(validation_times, pred_m_len)
            output_dic[pred_m] = df_output
        
        return output_dic

In [14]:
def export_scores_summary(output_dic, pred_ms = [1,2,3,4], filename='validation_score'):
    output_array = np.zeros((4, 2))
    for pred_m in pred_ms:
        df = output_dic[pred_m]
        output_array[pred_m-1] = df.groupby('scale')['smape'].mean().describe()[['mean', 'std']].to_numpy()

    df = pd.DataFrame(output_array, columns=['mean', 'std'], index=[1,2,3,4])
    df.to_csv(f'../output/{filename}.csv')

In [27]:
output_dic = run_validation(38, 5)
export_scores_summary(output_dic, pred_ms = [1], filename=f'lgbm_baseline_{USE_LAG}_{USE_OLD_LOG}_{USE_TREND}')

valid_times:  38
pred_m:  1
train_times:  37


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] lambda_l1 is set=1.8386216853616875, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8386216853616875
[LightGBM] [Warning] lambda_l2 is set=7.557660410418351, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.557660410418351
[LightGBM] [Warning] min_data_in_leaf is set=213, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=213
[LightGBM] [Warning] num_iterations is set=200, n_iter=200 will be ignored. Current value: num_iterations=200
# of cfips that have trend : 91
valid_times:  37
pred_m:  1
train_times:  36
[LightGBM] [Warning] lambda_l1 is set=1.8386216853616875, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8386216853616875
[LightGBM] [Warning] lambda_l2 is set=7.557660410418351, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.557660410418351
[LightGBM] [Warning] min_data_in_leaf is set=213, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=213
[LightGBM] [Warning] num_iterations

valid_times:  38
pred_m:  4
train_times:  34
[LightGBM] [Warning] lambda_l1 is set=1.8386216853616875, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8386216853616875
[LightGBM] [Warning] lambda_l2 is set=7.557660410418351, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.557660410418351
[LightGBM] [Warning] min_data_in_leaf is set=213, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=213
[LightGBM] [Warning] num_iterations is set=200, n_iter=200 will be ignored. Current value: num_iterations=200
valid_times:  37
pred_m:  4
train_times:  33
[LightGBM] [Warning] lambda_l1 is set=1.8386216853616875, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8386216853616875
[LightGBM] [Warning] lambda_l2 is set=7.557660410418351, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.557660410418351
[LightGBM] [Warning] min_data_in_leaf is set=213, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=213
[LightGBM] [Warning] n

In [31]:
utils.save_pickle(output_dic, 'result_lgbm_base')

## Create submission

In [14]:
def create_submission(filename=''):
    df_pred = run_validation_for_pred_m([39,40,41,42], [1,2,3,4])
    
    df_merged = pd.merge(df_subm, df_pred['mbd_pred'], how='left', on='row_id')
    df_merged.loc[~df_merged['mbd_pred'].isna(), 'microbusiness_density'] = df_merged['mbd_pred']
    df_submission = df_merged['microbusiness_density']
    
    if filename:
        df_submission.to_csv(f'../submission/{filename}.csv')
        print(f'saved {filename}')
        
    return df_pred, df_merged, df_submission

In [27]:
df_pred, df_merged, submission = create_submission(filename='lgbm_baseline_trend')

valid_times:  39
pred_m:  1
train_times:  38


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] lambda_l1 is set=1.8386216853616875, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8386216853616875
[LightGBM] [Warning] lambda_l2 is set=7.557660410418351, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.557660410418351
[LightGBM] [Warning] min_data_in_leaf is set=213, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=213
[LightGBM] [Warning] num_iterations is set=200, n_iter=200 will be ignored. Current value: num_iterations=200
# of cfips that have trend : 11
valid_times:  40
pred_m:  2
train_times:  38
[LightGBM] [Warning] lambda_l1 is set=1.8386216853616875, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.8386216853616875
[LightGBM] [Warning] lambda_l2 is set=7.557660410418351, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.557660410418351
[LightGBM] [Warning] min_data_in_leaf is set=213, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=213
[LightGBM] [Warning] num_iterations

### Ensemble with Best Score Code

In [15]:
df_sub_base = pd.read_csv('../submission/submission_10842.csv',  index_col='row_id')

In [16]:
pred_m=1
valid_time=39
trend_dict = get_trend_dict(valid_time, pred_m, 3, 3, 25000)
    
utils.save_pickle(trend_dict, 'trend_dict_3_3_25000')

In [17]:
for cfip in trend_dict:
    row_id = str(cfip) + '_2022-11-01'
    
    #　replace
    #　df_sub_base.loc[row_id, :] = (trend_dict[cfip] * df_all.loc[(df_all['scale']==38)&(df_all['cfips']==cfip), mbd]).values[0]
    
    trend_values = (trend_dict[cfip] * df_all.loc[(df_all['scale']==38)&(df_all['cfips']==cfip), mbd]).values[0]
    df_sub_base.loc[row_id, :] = (df_sub_base.loc[row_id].values[0] + trend_values) / 2
    
df_sub_base.to_csv('../submission/submission_10842_trend_ensemble.csv')